In [ ]:
!pip install -q streamlit
!pip install -q transformers==4.36.0
!pip install -p bitsandbytes==0.41.3 accelerate==0.26.0
!pip install -q -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 108.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 88.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.2.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.36.0 which is incompatible.

Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-opt

In [ ]:
%%capture
!pip install -q streamlit
!pip uninstall -y transformers
!pip install -q -U transformers

In [ ]:
!pip install -q accelerate
!pip install -q bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 17.4 MB/s eta 0:00:00


In [ ]:
# import streamlit as st
import numpy as np
from PIL import Image
import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration

In [ ]:
from transformers import BitsAndBytesConfig

nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
model_id = "llava-hf/llava-1.5-7b-hf"

processor = AutoProcessor.from_pretrained(model_id)

model = LlavaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.float32,
    low_cpu_mem_usage=True,
    load_in_4bit=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


processor_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

In [ ]:
# monkey image path
from PIL import Image, ImageDraw

monkey = '/content/drive/MyDrive/capstone_project/monkey.png'
img = Image.open(monkey)

# Resize the image to a smaller size
size = (150, 120)
img = img.resize(size)

In [ ]:
# for editing page format options
%%writefile recycling_app.py
import streamlit as st
from PIL import Image
import numpy as np
import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration
from sentence_transformers import SentenceTransformer
from sklearn.neighbors import NearestNeighbors
import pandas as pd

model_id = "llava-hf/llava-1.5-7b-hf"

processor = AutoProcessor.from_pretrained(model_id)

llava_model = LlavaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    load_in_4bit=True
)

encoder_model = SentenceTransformer('sentence-transformers/LaBSE')

## load RAG files

with open('/content/drive/My Drive/capstone_project/RAG_files/test_instructions.json', 'r') as openfile:

    recycling_instructions_open = json.load(openfile)

with open('/content/drive/My Drive/capstone_project/RAG_files/test_names_reference_list', 'rb') as dict_file:
    rag_embeddings = pickle.load(dict_file)

with open('/content/drive/My Drive/capstone_project/RAG_files/locations_idx_list', 'rb') as loc_dict_file:
    locations_json_idx_dict = pickle.load(loc_dict_file)

instruct_mapping_csv = pd.read_csv("/content/drive/My Drive/capstone_project/RAG_files/test_instruction_mapping_df.csv")

def retrive_instruct(class_choice, stage_1_output, location_name, recycling_instruction_dataset, locations_dict, instruc_mapping_df):

  locations_list = [x['location_id'] for x in recycling_instruction_dataset]

  loc_num = locations_dict[location_name]

  loc_idx = locations_list.index(loc_num)

  loc_dict = recycling_instruction_dataset[loc_idx]

  instructions_dict_list = loc_dict['instructions']


  relevant_sec_id = instruc_mapping_df[(instruc_mapping_df['pred_class']==class_choice) & (instruc_mapping_df['location_id']==loc_num)]['instruction_id'].values[0]
  stage_3_name =  instruc_mapping_df[(instruc_mapping_df['pred_class']==class_choice) & (instruc_mapping_df['location_id']==loc_num)]['generic names'].values[0]

  if stage_3_name == 'other':
    stage_3_name = stage_1_output

  instruc_id_list = [x['instruc_id'] for x in instructions_dict_list]

  instruc_idx = instruc_id_list.index(relevant_sec_id)

  retrieve_instruc = instructions_dict_list[instruc_idx]['instructions']

  return retrieve_instruc, stage_3_name

def knn_rag(stage_1_output, location, embeddings_cluster_dict, encoder_model):

  reference_words_list = list(embeddings_cluster_dict.keys())
  # encoder_model = SentenceTransformer('sentence-transformers/LaBSE')
  reference_embeddings = encoder_model.encode(reference_words_list)

  knn_model = NearestNeighbors(n_neighbors=1)
  knn_model.fit(reference_embeddings)

  query_phrase = stage_1_output.lower().strip(". ")

  query_llava = encoder_model.encode(query_phrase)

  dists, idxs = knn_model.kneighbors([query_llava])

  # print(idxs)
  neighbor = reference_words_list[idxs[0][0]]

  distance = dists[0]

  if distance <0.70:
    class_num = embeddings_cluster_dict[neighbor]
  else:
    class_num = 7

  instruction, stage_3_name = retrive_instruct(class_num, stage_1_output, location, recycling_instructions_open, locations_json_idx_dict, instruct_mapping_csv)

  return class_num, instruction, stage_3_name


def run_model(input_img, loc, processor, model, encoder_model):

  prompt = f"USER: <image>\nWhat is this waste item, what is it made of and primarily used for, don't consider the brand, output a short one phrase answer\nASSISTANT:"

  inputs = processor(prompt, input_img, return_tensors='pt').to(0, torch.float16)

  output = model.generate(**inputs, max_new_tokens=25, do_sample=False, temperature = 0)

  output_str = processor.decode(output[0][2:], skip_special_tokens=True)

  output_idx = output_str.index("ASSISTANT") + 11
  stage_1 = output_str[output_idx:]

  class_num, instruction, stage_3_name = knn_rag(stage_1, loc, rag_embeddings, encoder_model)

  prompt = f"USER: <image>\n Given these instructions: {instruction}. Is {stage_3_name} in the image recyclable Choose yes or no\nASSISTANT:"

  inputs = processor(prompt, input_img, return_tensors='pt').to(0, torch.float16)

  output = model.generate(**inputs, max_new_tokens=5, do_sample=False, temperature = 0)

  output_str = processor.decode(output[0][2:], skip_special_tokens=True)

  output_idx = output_str.index("ASSISTANT") + 11
  stage_2 = output_str[output_idx:]

  prompt = f"""USER: <image>\n Write recycling instructions based upon the following information about local recycling: {instruction}. Is this item able to be reyclced: {stage_2}. The descriptive name for the item is {stage_3_name}.
State in full sentences if this specific item can be recycled. Please provide instructions for how to recycle, create bulleted list instructions if it is recyclable, limit the answer to 3 sentences.\nASSISTANT:"""

  inputs = processor(prompt, input_img, return_tensors='pt').to(0, torch.float16)

  output = model.generate(**inputs, max_new_tokens=150, do_sample=False, temperature = 0)

  output_str = processor.decode(output[0][2:], skip_special_tokens=True)

  output_idx = output_str.index("ASSISTANT") + 11
  stage_3 = output_str[output_idx:]

  return stage_3

def model():
  custom_html = """
  <div class="container">
      <div class="image"> <img src="https://github.com/defoa2/Mids_capstone/blob/main/llava_jn_ad/monkey.png?raw=true" alt="This is my image." /> </div>
      <div class="text">
        <h1>Ask Jane: Our AI-Powered Recycling Assistant</h1>
      </div>
  </div>
  <style>
      .container {
        display: flex;
        gap: 50px;
        height: 300px;
        background-color: #84c477;
        }

        img {
          max-height: 50%;
        }

        .text {
          font-size: 18px;
        }
  </style>




  """
  # # Display the custom HTML
  st.components.v1.html(custom_html)
  st.subheader("About our model")
  st.write("Jane is a powerful Retrieval Augmented Generation (RAG) model that leverages both image classification and natural language generation to assist in the identification and proper presentation of items for recycling")
  st.write("Learn how to recycle with the recycling assistant Jane")

  st.subheader('Ask your recycling question')
  st.write("Enter your location and take a picture of something that you want to throw away and upload it to learn how to recycle")

  upload_col1, upload_col2 = st.columns([1,1])
  with upload_col1:
    # st.image("https://github.com/defoa2/Mids_capstone/blob/main/llava_jn_ad/recyclingImage2.png?raw=true")
    loc_option = st.selectbox(
    "What is your location?",
    ("Palm Desert", "Cook County"),
    )
    image = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])

  img_exists = False

  with upload_col2:
    if image is not None:
      image_input = Image.open(image)
      img_exists = True
      st.write("Your image:")
      st.image(image)

      # if st.button("Create Instructions"):
      #   model_output = run_model(image_input, loc_option)
      # st.write("The instructions for how to recycle:")
      # #   st.write(model_output)
      # st.write("Paper cups can not be recycled in your location, as they have a plastic lining that prevents proper recycling. Please dispose of it in the nearest trash can.")

    else:
      st.write("Please upload an image")

  button_col1, button_col2, button_col3 = st.columns([1,1,1])
  with button_col2:
    if st.button("Ask Jane"):
      model_output = run_model(image_input, loc_option, processor, llava_model, encoder_model)


      st.subheader("Smart recycling instructions")

      style_str = """
      <style>
          .container {
            display: flex;
            justify-content: center;
            align-items: center;
            background-color: #F5F5F5;
            }
          .text {
              font-size: 20px;
              text-align: left;
              width: 90%
            }
      </style>

      """

      instruct_html = f"""
      <div class="container">
          <div class="text">
            <p> <B>Jane: </B> {model_output}</p>
          </div>
      </div>
      {style_string}


      """
      st.components.v1.html(instruct_html)
       # st.write(model_output)

  # st.set_page_config(layout="wide")

  # #img = Image.open('/content/drive/My Drive/capstone_project/monkey.png')
  # #size = (150, 120)
  # #img = img.resize(size)

  # img = Image.open('/content/drive/My Drive/capstone_project/Title1.png')
  # st.image(img)

  # st.write("")
  # st.write("Jane is a powerful Retrieval-Augmented Generation (RAG) model that leverages both image classification and natural language generation to assist in the identification and proper preparation of items for recycling.")
  # st.write("")

  # col1, col2 = st.columns(2, gap="large")

  # with col1:
  #   loc_option = st.selectbox(
  #     "Select your location",
  #     ("Palm Desert", "Cook County"),
  #   )

  # with col2:
  #   st.write("Snap a photo of an item you want to dispose of and upload it to learn how to recycle it properly.")
  #   img_exists = False
  #   image = st.file_uploader("Upload an image (jpg, jpeg, or png)", type=["jpg", "jpeg", "png"])
  #   if image is not None:
  #     image_input = Image.open(image)
  #     img_exists = True
  #     st.image(image)
  #   # st.button("Create Instructions")
  #     if st.button("Ask Jane!"): #("Get Disposal Instructions")
  #       model_output = run_model(image_input, loc_option)
  #       st.write("Instructions:")
  #       st.write(model_output)
  #   else:
  #     st.write("Please upload an image")

pg = st.navigation([st.Page(model)])
pg.run()

Overwriting recycling_app.py


In [ ]:
!mkdir -p .streamlit
!echo "[theme]" >> .streamlit/config.toml
!echo "primaryColor = \"#84c477\"" >> .streamlit/config.toml
!echo "backgroundColor = \"#FFFFFF\"" >> .streamlit/config.toml
!echo "secondaryBackgroundColor = \"#F0F2F6\"" >> .streamlit/config.toml
!echo "textColor = \"#262730\"" >> .streamlit/config.toml
!echo "font = \"sans serif\"" >> .streamlit/config.toml

### Launch Streamlit

In [ ]:
!npm install localtunnel


up to date, audited 23 packages in 475ms

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [ ]:
!streamlit run recycling_app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.46.195.247
your url is: https://ripe-poets-heal.loca.lt


In [ ]:
# if 'recent_output' not in st.session_state:
#     st.session_state['recent_output'] = 0

# def retrive_instruct(class_choice, location_name):
#   recycling_instruction_dataset = [
#     {'location_id':1,
#     'location':"Cook County, IL",
#      'instructions': [
#          {
#          "instruc_id":0,
#          "items": "metal",
#          "instructions": "Metal objects that are able to be recycled includes steel and aluminum containers and foil."
#         },
#          {"instruc_id":1,
#          "items": "paper",
#          "instructions": "paper that can be recycled includes cardboard (flattened), office paper, newspaper, and magazines"
#           },
#          {
#           "instruc_id":2,
#          "items": "glass",
#          "instructions": "glass items that can be recycled includes glass bottles and jars only"
#          },
#          {
#           "instruc_id":3,
#          "items": "plastic",
#          "instructions": """Plastic items that can be recycled includes plastic bottles, tubs, jugs, and jars only.
#           Always replace your cap onto the empty and clean plastic bottle it came from.
#           Then, place into your recycling bin to ensure both plastic components are successfully processed at your local recycling facility."""
#          },
#          {
#           "instruc_id":4,
#          "items": "coffee cups",
#          "instructions": """Your cup of joe is a recycling no. Lids, straws, and hot beverage cups should be kept out of your curbside recycling bin.
#           Despite the paper composition of hot beverage cups, they contain a lining that degrades the overall quality of the recycling batch and should not be recycled in your bin.
#           For iced coffee, some uncertainty remains about whether the plastic cup is recyclable. Please check with your local authority to determine whether plastic coffee cups can go in your bin.
#           To reduce waste and save resources, use a reusable coffee cup and straw! Some coffee establishments even offer discounts when you bring your own cup/mug."""
#          },
#          {
#           "instruc_id":5,
#          "items": "other",
#          "instructions": "This item is not included in the this of acceptable recyclables, please dispose of it in the garbage."
#          }
#          ]
#      },
#     {
#       'location_id':2,
#       'location':"Palm Desert, CA",
#       'instructions': [
#          {
#          "instruc_id":0,
#          "items": "metal and glass",
#          "instructions": """The following metal and glass items can be recycled:
#          -Aluminum, steel, and tin cans,
#          -Clean aluminum foil,
#         - Glass bottles and jars (remove lids)"""
#         },
#          {"instruc_id":1,
#          "items": "paper",
#          "instructions": """
#          The following paper items can be recycled:
#         - White and colored paper,
#         - Phone books,
#         - Magazines,
#         - Junk Mail,
#         - Envelopes,
#         - File folders,
#         - Cardboard (flattened),
#         - Cartons (milk, juice, & broth),
#         - Books (remove hard covers or binding),
#         - Cardboard boxes, e.g. cereal, etc.
#          """
#           },
#          {
#           "instruc_id":2,
#          "items": "plastic",
#          "instructions": """Plastic items that can be recycled includes:
#           - Styrofoam packing blocks
#           - Plastic medicine bottles (empty)
#           - Plastic bottles numbered 1-7 (remove plastic film or wrap)"""
#          },
#          {
#           "instruc_id":3,
#          "items": "other naps",
#          "instructions": "This item is not included in the this of acceptable recyclables, please dispose of it in the garbage."
#          }
#          ]
#     }
#   ]

#   locations_list = [x['location_id'] for x in recycling_instruction_dataset]

#   locations_dict = {"Palm Desert":2, "Cook County":1}
#   loc_num = locations_dict[location_name]

#   loc_idx = locations_list.index(loc_num)

#   loc_dict = recycling_instruction_dataset[loc_idx]

#   instructions_dict_list = loc_dict['instructions']


#   classes_col = np.repeat(np.arange(1, 8), 2)
#   loc_name = np.tile(['Cook County, IL', 'Palm Desert, CA'], 7)
#   loc_idx = np.tile([1, 2], 7)
#   mapping_to_instruc = [0, 0, 0, 0, 4, 3, 1, 1, 2, 0, 3, 2, 5, 3]

#   instruc_mapping_df = pd.DataFrame({'pred_class': classes_col, 'location': loc_name, 'location_id': loc_idx, 'instruction_id': mapping_to_instruc})
#   relevant_sec_id = instruc_mapping_df[(instruc_mapping_df['pred_class']==class_choice) & (instruc_mapping_df['location_id']==loc_num)]['instruction_id'].values[0]
#   instruc_id_list = [x['instruc_id'] for x in instructions_dict_list]

#   instruc_idx = instruc_id_list.index(relevant_sec_id)

#   retrieve_instruc = instructions_dict_list[instruc_idx]['instructions']

#   return retrieve_instruc

# def knn_rag(stage_1_output, location):
#   embeddings_cluster_dict = {"aluminum food can":1,
#               "tin can":1,
#               "steel can":1,
#               "canned food container":1,
#               "canned food can":1,
#               "soda can":2,
#               "beverage can":2,
#               "soft drink can":2,
#               "pepsi can":2,
#               "coca cola can":2,
#               "paper cup":3,
#               "hot beverage cup":3,
#               "coffee cup":3,
#               "lined paper cup":3,
#               "disposable coffee cup":3,
#               "white paper cup":3,
#               "cardboard":4,
#              "cardboard box":4,
#               "box":4,
#               "packing box":4,
#               "cereal box":4,
#               "glass bottle":5,
#               "glass drink container":5,
#               "alcohol bottle":5,
#               "beer bottle":5,
#               "water bottle":6,
#               "disposable water bottle":6,
#               "plastic bottle":6,
#               "plastic soda bottle":6}

#   reference_words_list = list(embeddings_cluster_dict.keys())
#   encoder_model = SentenceTransformer('sentence-transformers/LaBSE')
#   reference_embeddings = encoder_model.encode(reference_words_list)

#   knn_model = NearestNeighbors(n_neighbors=1)
#   knn_model.fit(reference_embeddings)

#   query_llava = encoder_model.encode(stage_1_output)

#   dists, idxs = knn_model.kneighbors([query_llava])

#   neighbor = reference_words_list[idxs[0][0]]

#   distance = dists[0]

#   if distance <0.9:
#     class_num = embeddings_cluster_dict[neighbor]
#   else:
#     class_num = 7

#   instruction = retrive_instruct(class_num, location)

#   return class_num, instruction


# def run_model(input_img, loc):
#   # model_id = "llava-hf/llava-1.5-7b-hf"

#   # processor = AutoProcessor.from_pretrained(model_id)

#   # model = LlavaForConditionalGeneration.from_pretrained(
#   #     model_id,
#   #     torch_dtype=torch.float16,
#   #     low_cpu_mem_usage=True,
#   #     load_in_4bit=True
#   # )
#   prompt = f"USER: <image>\nWhat is this waste item and what material is the waste item made from, output a one phrase answer, don't consider the brand\nASSISTANT:"

#   inputs = processor(prompt, input_img, return_tensors='pt').to(0, torch.float16)

#   output = llava_model.generate(**inputs, max_new_tokens=25, do_sample=False, temperature = 0)

#   output_str = processor.decode(output[0][2:], skip_special_tokens=True)

#   output_idx = output_str.index("ASSISTANT") + 11
#   stage_1 = output_str[output_idx:]

#   class_num, instruction = knn_rag(stage_1, loc)

#   prompt = f"USER: <image>\n Given these instructions: {instruction}. Is {stage_1} in the image recyclable Choose yes or no\nASSISTANT:"

#   inputs = processor(prompt, input_img, return_tensors='pt').to(0, torch.float16)

#   output = llava_model.generate(**inputs, max_new_tokens=5, do_sample=False, temperature = 0)

#   output_str = processor.decode(output[0][2:], skip_special_tokens=True)

#   output_idx = output_str.index("ASSISTANT") + 11
#   stage_2 = output_str[output_idx:]

#   prompt = f"USER: <image>\n Given these instructions: {instruction}. Is this item able to be reyclced: {stage_2}. Please give instructions on how to recycle item in image\nASSISTANT:"

#   inputs = processor(prompt, input_img, return_tensors='pt').to(0, torch.float16)

#   output = llava_model.generate(**inputs, max_new_tokens=50, do_sample=False, temperature = 0)

#   output_str = processor.decode(output[0][2:], skip_special_tokens=True)

#   output_idx = output_str.index("ASSISTANT") + 11
#   stage_3 = output_str[output_idx:]

#   return stage_3

# def get_image():

#   # example input image
#   image_arr = np.load("/content/drive/My Drive/capstone_project/image_data/data_x.npy", allow_pickle = True)
#   test_im = Image.fromarray(np.uint8(image_arr[20]))

#   # output = run_model(test_im)
#   return test_im

# def home():
#   st.title("Home")
#   st.write("This is the home page")

In [ ]:
!

### test for only UI

In [ ]:
%%writefile recycling_app_test_ui.py
import streamlit as st
from PIL import Image
import numpy as np
import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration
from sentence_transformers import SentenceTransformer
from sklearn.neighbors import NearestNeighbors
import pandas as pd

def get_image():
  # example input image
  image_arr = np.load("/content/drive/My Drive/capstone_project/image_data/data_x.npy", allow_pickle = True)
  test_im = Image.fromarray(np.uint8(image_arr[20]))

  # output = run_model(test_im)
  return test_im

def home():
  st.title("Home")
  st.write("This is the home page")

def recycling_hub():
  st.title("Recycle Right: Smart Recycling Model")
  st.write("Learn how to recycle with the recycling assistant Jane")
  recycling_image_monkey = Image.open("/content/drive/My Drive/capstone_project/monkey.png")
  st.image(recycling_image_monkey)
  loc_option = st.selectbox(
    "What is your location?",
    ("Palm Desert", "Cook County"),
)
  # image_to_display = get_image()
  # model, processor = load_model()
  st.write("Take a picture of something that you want to throw away and input to learn how to recycle")
  img_exists = False
  image = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])
  if image is not None:
    image_input = Image.open(image)
    img_exists = True
    st.image(image)
  # st.button("Create Instructions")
    # if st.button("Create Instructions"):
    #   model_output = run_model(image_input, loc_option)
    st.write("The instructions for how to recycle:")
    #   st.write(model_output)
    st.write("Aluminum cans can be recycled in Palm Desert, CA. Please dispose of it in the nearest recycling")

  else:
    st.write("Please upload an image")

# def results():
#   st.title("Find your recycling instructions here!")

#   if st.session_state['recent_output'] != 0:
#     st.write("Here are your recycling instructions: \n")
#     st.write(st.session_state['recent_output'])

#   else:
#     st.write("Please run the model to get your instructions")

def about():
  st.title("About")
  st.write("This is the about page")

pg = st.navigation([st.Page(home), st.Page(recycling_hub), st.Page(about)])
pg.run()

In [ ]:
!npm install localtunnel

In [ ]:
!streamlit run recycling_app_test_ui.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

## test outside of streamlit

In [ ]:
import streamlit as st
from PIL import Image
import numpy as np
import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration
from sentence_transformers import SentenceTransformer
from sklearn.neighbors import NearestNeighbors
import pandas as pd



def retrive_instruct(class_choice, location_name):
  recycling_instruction_dataset = [
    {'location_id':1,
    'location':"Cook County, IL",
     'instructions': [
         {
         "instruc_id":0,
         "items": "metal",
         "instructions": "Metal objects that are able to be recycled includes steel and aluminum containers and foil."
        },
         {"instruc_id":1,
         "items": "paper",
         "instructions": "paper that can be recycled includes cardboard (flattened), office paper, newspaper, and magazines"
          },
         {
          "instruc_id":2,
         "items": "glass",
         "instructions": "glass items that can be recycled includes glass bottles and jars only"
         },
         {
          "instruc_id":3,
         "items": "plastic",
         "instructions": """Plastic items that can be recycled includes plastic bottles, tubs, jugs, and jars only.
          Always replace your cap onto the empty and clean plastic bottle it came from.
          Then, place into your recycling bin to ensure both plastic components are successfully processed at your local recycling facility."""
         },
         {
          "instruc_id":4,
         "items": "coffee cups",
         "instructions": """Your cup of joe is a recycling no. Lids, straws, and hot beverage cups should be kept out of your curbside recycling bin.
          Despite the paper composition of hot beverage cups, they contain a lining that degrades the overall quality of the recycling batch and should not be recycled in your bin.
          For iced coffee, some uncertainty remains about whether the plastic cup is recyclable. Please check with your local authority to determine whether plastic coffee cups can go in your bin.
          To reduce waste and save resources, use a reusable coffee cup and straw! Some coffee establishments even offer discounts when you bring your own cup/mug."""
         },
         {
          "instruc_id":5,
         "items": "other",
         "instructions": "This item is not included in the this of acceptable recyclables, please dispose of it in the garbage."
         }
         ]
     },
    {
      'location_id':2,
      'location':"Palm Desert, CA",
      'instructions': [
         {
         "instruc_id":0,
         "items": "metal and glass",
         "instructions": """The following metal and glass items can be recycled:
         -Aluminum, steel, and tin cans,
         -Clean aluminum foil,
        - Glass bottles and jars (remove lids)"""
        },
         {"instruc_id":1,
         "items": "paper",
         "instructions": """
         The following paper items can be recycled:
        - White and colored paper,
        - Phone books,
        - Magazines,
        - Junk Mail,
        - Envelopes,
        - File folders,
        - Cardboard (flattened),
        - Cartons (milk, juice, & broth),
        - Books (remove hard covers or binding),
        - Cardboard boxes, e.g. cereal, etc.
         """
          },
         {
          "instruc_id":2,
         "items": "plastic",
         "instructions": """Plastic items that can be recycled includes:
          - Styrofoam packing blocks
          - Plastic medicine bottles (empty)
          - Plastic bottles numbered 1-7 (remove plastic film or wrap)"""
         },
         {
          "instruc_id":3,
         "items": "other naps",
         "instructions": "This item is not included in the this of acceptable recyclables, please dispose of it in the garbage."
         }
         ]
    }
  ]

  locations_list = [x['location_id'] for x in recycling_instruction_dataset]

  locations_dict = {"Palm Desert":2, "Cook County":1}
  loc_num = locations_dict[location_name]

  loc_idx = locations_list.index(loc_num)

  loc_dict = recycling_instruction_dataset[loc_idx]

  instructions_dict_list = loc_dict['instructions']


  classes_col = np.repeat(np.arange(1, 8), 2)
  loc_name = np.tile(['Cook County, IL', 'Palm Desert, CA'], 7)
  loc_idx = np.tile([1, 2], 7)
  mapping_to_instruc = [0, 0, 0, 0, 4, 3, 1, 1, 2, 0, 3, 2, 5, 3]

  instruc_mapping_df = pd.DataFrame({'pred_class': classes_col, 'location': loc_name, 'location_id': loc_idx, 'instruction_id': mapping_to_instruc})
  relevant_sec_id = instruc_mapping_df[(instruc_mapping_df['pred_class']==class_choice) & (instruc_mapping_df['location_id']==loc_num)]['instruction_id'].values[0]
  instruc_id_list = [x['instruc_id'] for x in instructions_dict_list]

  instruc_idx = instruc_id_list.index(relevant_sec_id)

  retrieve_instruc = instructions_dict_list[instruc_idx]['instructions']

  return retrieve_instruc

def knn_rag(stage_1_output, location):
  embeddings_cluster_dict = {"aluminum food can":1,
              "tin can":1,
              "steel can":1,
              "canned food container":1,
              "canned food can":1,
              "soda can":2,
              "beverage can":2,
              "soft drink can":2,
              "pepsi can":2,
              "coca cola can":2,
              "paper cup":3,
              "hot beverage cup":3,
              "coffee cup":3,
              "lined paper cup":3,
              "disposable coffee cup":3,
              "white paper cup":3,
              "cardboard":4,
             "cardboard box":4,
              "box":4,
              "packing box":4,
              "cereal box":4,
              "glass bottle":5,
              "glass drink container":5,
              "alcohol bottle":5,
              "beer bottle":5,
              "water bottle":6,
              "disposable water bottle":6,
              "plastic bottle":6,
              "plastic soda bottle":6}

  reference_words_list = list(embeddings_cluster_dict.keys())
  encoder_model = SentenceTransformer('sentence-transformers/LaBSE')
  reference_embeddings = encoder_model.encode(reference_words_list)

  knn_model = NearestNeighbors(n_neighbors=1)
  knn_model.fit(reference_embeddings)

  query_llava = encoder_model.encode(stage_1_output)

  dists, idxs = knn_model.kneighbors([query_llava])

  print(idxs)
  neighbor = reference_words_list[idxs[0][0]]

  distance = dists[0]

  if distance <0.9:
    class_num = embeddings_cluster_dict[neighbor]
  else:
    class_num = 7

  instruction = retrive_instruct(class_num, location)

  return class_num, instruction


def run_model(input_img, loc):
  model_id = "llava-hf/llava-1.5-7b-hf"

  processor = AutoProcessor.from_pretrained(model_id)

  model = LlavaForConditionalGeneration.from_pretrained(
      model_id,
      torch_dtype=torch.float16,
      low_cpu_mem_usage=True,
      load_in_4bit=True
  )
  prompt = f"USER: <image>\nWhat is this waste item and what material is the waste item made from, output a one phrase answer, don't consider the brand\nASSISTANT:"

  inputs = processor(prompt, input_img, return_tensors='pt').to(0, torch.float16)

  output = model.generate(**inputs, max_new_tokens=25, do_sample=False, temperature = 0)

  output_str = processor.decode(output[0][2:], skip_special_tokens=True)

  output_idx = output_str.index("ASSISTANT") + 11
  stage_1 = output_str[output_idx:]

  class_num, instruction = knn_rag(stage_1, loc)

  prompt = f"USER: <image>\n Given these instructions: {instruction}. Is {stage_1} in the image recyclable Choose yes or no\nASSISTANT:"

  inputs = processor(prompt, input_img, return_tensors='pt').to(0, torch.float16)

  output = model.generate(**inputs, max_new_tokens=5, do_sample=False, temperature = 0)

  output_str = processor.decode(output[0][2:], skip_special_tokens=True)

  output_idx = output_str.index("ASSISTANT") + 11
  stage_2 = output_str[output_idx:]

  prompt = f"USER: <image>\n Given these instructions: {instruction}. Is this item able to be reyclced: {stage_2}. Please give instructions on how to recycle item in image\nASSISTANT:"

  inputs = processor(prompt, input_img, return_tensors='pt').to(0, torch.float16)

  output = model.generate(**inputs, max_new_tokens=50, do_sample=False, temperature = 0)

  output_str = processor.decode(output[0][2:], skip_special_tokens=True)

  output_idx = output_str.index("ASSISTANT") + 11
  stage_3 = output_str[output_idx:]

  return stage_3


loc_option = "Palm Desert"
image_arr = np.load("/content/drive/My Drive/capstone_project/image_data/data_x.npy", allow_pickle = True)
image_to_display = Image.fromarray(np.uint8(image_arr[20]))

model_output = run_model(image_to_display, loc_option)
print(model_output)

In [ ]:
import streamlit as st
from PIL import Image
import numpy as np
import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration
from sentence_transformers import SentenceTransformer
from sklearn.neighbors import NearestNeighbors
import pandas as pd

model_id = "llava-hf/llava-1.5-7b-hf"

processor = AutoProcessor.from_pretrained(model_id)

model = LlavaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    load_in_4bit=True
)

def retrive_instruct(class_choice, location_name):
  recycling_instruction_dataset = [
    {'location_id':1,
    'location':"Cook County, IL",
     'instructions': [
         {
         "instruc_id":0,
         "items": "metal",
         "instructions": "Metal objects that are able to be recycled includes steel and aluminum containers and foil."
        },
         {"instruc_id":1,
         "items": "paper",
         "instructions": "paper that can be recycled includes cardboard (flattened), office paper, newspaper, and magazines"
          },
         {
          "instruc_id":2,
         "items": "glass",
         "instructions": "glass items that can be recycled includes glass bottles and jars only"
         },
         {
          "instruc_id":3,
         "items": "plastic",
         "instructions": """Plastic items that can be recycled includes plastic bottles, tubs, jugs, and jars only.
          Always replace your cap onto the empty and clean plastic bottle it came from.
          Then, place into your recycling bin to ensure both plastic components are successfully processed at your local recycling facility."""
         },
         {
          "instruc_id":4,
         "items": "coffee cups",
         "instructions": """Your cup of joe is a recycling no. Lids, straws, and hot beverage cups should be kept out of your curbside recycling bin.
          Despite the paper composition of hot beverage cups, they contain a lining that degrades the overall quality of the recycling batch and should not be recycled in your bin.
          For iced coffee, some uncertainty remains about whether the plastic cup is recyclable. Please check with your local authority to determine whether plastic coffee cups can go in your bin.
          To reduce waste and save resources, use a reusable coffee cup and straw! Some coffee establishments even offer discounts when you bring your own cup/mug."""
         },
         {
          "instruc_id":5,
         "items": "other",
         "instructions": "This item is not included in the this of acceptable recyclables, please dispose of it in the garbage."
         }
         ]
     },
    {
      'location_id':2,
      'location':"Palm Desert, CA",
      'instructions': [
         {
         "instruc_id":0,
         "items": "metal and glass",
         "instructions": """The following metal and glass items can be recycled:
         -Aluminum, steel, and tin cans,
         -Clean aluminum foil,
        - Glass bottles and jars (remove lids)"""
        },
         {"instruc_id":1,
         "items": "paper",
         "instructions": """
         The following paper items can be recycled:
        - White and colored paper,
        - Phone books,
        - Magazines,
        - Junk Mail,
        - Envelopes,
        - File folders,
        - Cardboard (flattened),
        - Cartons (milk, juice, & broth),
        - Books (remove hard covers or binding),
        - Cardboard boxes, e.g. cereal, etc.
         """
          },
         {
          "instruc_id":2,
         "items": "plastic",
         "instructions": """Plastic items that can be recycled includes:
          - Styrofoam packing blocks
          - Plastic medicine bottles (empty)
          - Plastic bottles numbered 1-7 (remove plastic film or wrap)"""
         },
         {
          "instruc_id":3,
         "items": "other naps",
         "instructions": "This item is not included in the this of acceptable recyclables, please dispose of it in the garbage."
         }
         ]
    }
  ]

  locations_list = [x['location_id'] for x in recycling_instruction_dataset]

  locations_dict = {"Palm Desert":2, "Cook County":1}
  loc_num = locations_dict[location_name]

  loc_idx = locations_list.index(loc_num)

  loc_dict = recycling_instruction_dataset[loc_idx]

  instructions_dict_list = loc_dict['instructions']


  classes_col = np.repeat(np.arange(1, 8), 2)
  loc_name = np.tile(['Cook County, IL', 'Palm Desert, CA'], 7)
  loc_idx = np.tile([1, 2], 7)
  mapping_to_instruc = [0, 0, 0, 0, 4, 3, 1, 1, 2, 0, 3, 2, 5, 3]

  instruc_mapping_df = pd.DataFrame({'pred_class': classes_col, 'location': loc_name, 'location_id': loc_idx, 'instruction_id': mapping_to_instruc})
  relevant_sec_id = instruc_mapping_df[(instruc_mapping_df['pred_class']==class_choice) & (instruc_mapping_df['location_id']==loc_num)]['instruction_id'].values[0]
  instruc_id_list = [x['instruc_id'] for x in instructions_dict_list]

  instruc_idx = instruc_id_list.index(relevant_sec_id)

  retrieve_instruc = instructions_dict_list[instruc_idx]['instructions']

  return retrieve_instruc

def knn_rag(stage_1_output, location):
  embeddings_cluster_dict = {"aluminum food can":1,
              "tin can":1,
              "steel can":1,
              "canned food container":1,
              "canned food can":1,
              "soda can":2,
              "beverage can":2,
              "soft drink can":2,
              "pepsi can":2,
              "coca cola can":2,
              "paper cup":3,
              "hot beverage cup":3,
              "coffee cup":3,
              "lined paper cup":3,
              "disposable coffee cup":3,
              "white paper cup":3,
              "cardboard":4,
             "cardboard box":4,
              "box":4,
              "packing box":4,
              "cereal box":4,
              "glass bottle":5,
              "glass drink container":5,
              "alcohol bottle":5,
              "beer bottle":5,
              "water bottle":6,
              "disposable water bottle":6,
              "plastic bottle":6,
              "plastic soda bottle":6}

  reference_words_list = list(embeddings_cluster_dict.keys())
  encoder_model = SentenceTransformer('sentence-transformers/LaBSE')
  reference_embeddings = encoder_model.encode(reference_words_list)

  knn_model = NearestNeighbors(n_neighbors=1)
  knn_model.fit(reference_embeddings)

  query_llava = encoder_model.encode(stage_1_output)

  dists, idxs = knn_model.kneighbors([query_llava])

  neighbor = reference_words_list[idxs[0][0]]

  distance = dists[0]

  if distance <0.9:
    class_num = embeddings_cluster_dict[neighbor]
  else:
    class_num = 7

  instruction = retrive_instruct(class_num, location)

  return class_num, instruction


def run_model(input_img, loc):
  # model_id = "llava-hf/llava-1.5-7b-hf"

  # processor = AutoProcessor.from_pretrained(model_id)

  # model = LlavaForConditionalGeneration.from_pretrained(
  #     model_id,
  #     torch_dtype=torch.float16,
  #     low_cpu_mem_usage=True,
  #     load_in_4bit=True
  # )
  prompt = f"USER: <image>\nWhat is this waste item and what material is the waste item made from, output a one phrase answer, don't consider the brand\nASSISTANT:"

  inputs = processor(prompt, input_img, return_tensors='pt').to(0, torch.float16)

  output = model.generate(**inputs, max_new_tokens=25, do_sample=False, temperature = 0)

  output_str = processor.decode(output[0][2:], skip_special_tokens=True)

  output_idx = output_str.index("ASSISTANT") + 11
  stage_1 = output_str[output_idx:]

  class_num, instruction = knn_rag(stage_1, loc)

  prompt = f"USER: <image>\n Given these instructions: {instruction}. Is {stage_1} in the image recyclable Choose yes or no\nASSISTANT:"

  inputs = processor(prompt, input_img, return_tensors='pt').to(0, torch.float16)

  output = model.generate(**inputs, max_new_tokens=5, do_sample=False, temperature = 0)

  output_str = processor.decode(output[0][2:], skip_special_tokens=True)

  output_idx = output_str.index("ASSISTANT") + 11
  stage_2 = output_str[output_idx:]

  prompt = f"USER: <image>\n Given these instructions: {instruction}. Is this item able to be reyclced: {stage_2}. Please give instructions on how to recycle item in image\nASSISTANT:"

  inputs = processor(prompt, input_img, return_tensors='pt').to(0, torch.float16)

  output = model.generate(**inputs, max_new_tokens=50, do_sample=False, temperature = 0)

  output_str = processor.decode(output[0][2:], skip_special_tokens=True)

  output_idx = output_str.index("ASSISTANT") + 11
  stage_3 = output_str[output_idx:]

  return stage_3

loc_option = "Palm Desert"
image_arr = np.load("/content/drive/My Drive/capstone_project/image_data/data_x.npy", allow_pickle = True)
image_to_display = Image.fromarray(np.uint8(image_arr[20]))

model_output = run_model(image_to_display, loc_option)
print(model_output)


In [ ]:
test_list = [[0]]
test_list[0]

In [ ]:
To recycle the aluminum can in the image, follow these steps:

1. Remove the label from the can.
2. Rinse the can with warm water to remove any residue or leftover food.